In [1]:
# Step1: Generate each_contest_before_csv
import xml.etree.ElementTree as ET
import pandas as pd  

# Input will be the card_ID 
cols = ["卡號","姓名",  "車隊", "車型", "日期"]
rows = []

#Parse XML file
tree = ET.parse('race_cars_info.xml')
root = tree.getroot()
for elem in root:
        name = elem.find("姓名").text
        car_label = elem.find("卡號").text
        car_team = elem.find("車隊").text
        car_model = elem.find("車型").text
        date = elem.find("日期").text 
            
        rows.append({"姓名": name, 
                  "卡號": car_label,
                  "姓名": name,
                  "車隊": car_team,
                  "車型": car_model,
                  "日期": date})

df = pd.DataFrame(rows, columns = cols) 
  
# # write dataframe to csv
df.to_csv('each_contest_before.csv',index=False)

key = cardId = string = "1014E3_op"       --> value = 車號


In [2]:
df

,卡號,姓名,車隊,車型,日期
0,#1,Steven,team1,BMW M4,2019-03-02 11:16:07
1,#2,Michael,team2,McLaren 720S,2019-06-01 10:06:07
2,#3,Albert,team3,Porsche 918 GTR,2019-04-05 16:30:07


In [3]:
#1. Insert each personName with corresponding cardID
# Need All people's cardIDs!!!

cardIDMap = {}
cardIDMap["1014E3"] = "#1"        # Steven
cardIDMap["1014E9"] = "#2"        # Albert
cardIDMap["10155D"] = "#3"        # Michael 


import math
bestScoreMap = {}
bestScoreMap["1014E3"] = math.inf         # Steven
bestScoreMap["1014E9"] = math.inf         # Albert
bestScoreMap["10155D"] = math.inf        # Michael 

In [4]:
print(cardIDMap)

{'1014E3': '#1', '1014E9': '#2', '10155D': '#3'}


In [5]:
print(bestScoreMap)

{'1014E3': inf, '1014E9': inf, '10155D': inf}


# 模擬三張不同的卡號 給予六圈的紀錄（修改前面10 個字數的絕對時間點）


----------------- 1014E3_op（卡號 ＃1) ----------------- 此卡號給steven


1651545100 472058 2022/05/03 Tue - 10:24:00.472


1651545200 686995 2022/05/03 Tue - 10:32:09.686


1651545300 408053 2022/05/03 Tue - 10:34:14.408


1651545400 79984 2022/05/03 Tue - 10:38:14.079


1651545500 802198 2022/05/03 Tue - 10:42:09.802


1651545600 426224 2022/05/03 Tue - 10:43:05.426





----------------- 1014E9_op（卡號 ＃2)----------------- 此卡號給Albert


1651545100 472058 2022/05/03 Tue - 10:33:00.472


1651545300 686995 2022/05/03 Tue - 10:35:09.686


1651545450 408053 2022/05/03 Tue - 10:37:14.408


1651545500 79984 2022/05/03 Tue - 10:39:14.079


1651545670 802198 2022/05/03 Tue - 10:41:10.802


1651545850 426224 2022/05/03 Tue - 10:43:05.426



----------------- 10155D_op（卡號 ＃3) ----------------- 此車號給Michael


1651545180 472058 2022/05/03 Tue - 10:33:00.472


1651545309 686995 2022/05/03 Tue - 10:35:09.686








In [6]:
# Input = mm
# Output = nn:sec.mm
def formattedTime(timeStamp):
    
    msPerMin = 60*1000

    minutes = math.trunc(timeStamp/(msPerMin))
    seconds = math.trunc((timeStamp - (minutes * msPerMin)) / 1000); 
    miliSeconds = timeStamp % 1000; 

    minutes = str(minutes).rjust(2,"0")
    seconds = str(seconds).rjust(2,"0")
    miliSeconds = str(miliSeconds).rjust(3,"0")
            
    return (minutes + ":" + seconds + "." + miliSeconds)

In [7]:
def readCSVAndparseOpFiles():

    df = pd.read_csv ('each_contest_before.csv')
    df['車競賽'] = 1

    #1. Mapping the card ID with its corresponding timestamps. 

    #2. How to parse the filename with op extension? 

    #3. How to keep updating the latest records within 1 to 10 sec. 


    #4. Calculate time traveling 

    import glob, os


    # Please change path to be the current directory. 
    os.chdir("/Users/stevenkan/Documents/Project/Race_Car_Stream")


    for cardIDfileName in glob.glob("*op"):

        time = open(cardIDfileName).readlines()
        cardID = cardIDfileName[:6]
        
        # Skip If cardID map doesn't contain the given key
        if cardID not in cardIDMap:
            print(cardID + "_op 不在此車競賽! 不做任何分析")
            continue
        
        personID = cardIDMap[cardID]

        # Extract each timestamp in each line. 
        i = 0
        prev = 0
        for line in time:
            timestamp = int(line[:10])
    #         print(timestamp)

            index = df.index
            new_val = df['卡號'] == personID
            index = index[new_val]

            timeTravel = timestamp - prev
            bestScoreMap[cardID] = min(bestScoreMap[cardID], timeTravel)
            
            
            
            formattedTimeTravel = formattedTime(timeTravel)
        
            if(i == 1):
                df.loc[index, '第一圈秒數'] = formattedTimeTravel
            elif(i == 2):
                df.loc[index, '第二圈秒數'] = formattedTimeTravel
            elif(i == 3):
                df.loc[index, '第三圈秒數'] = formattedTimeTravel
            elif(i == 4):
                df.loc[index, '第四圈秒數'] = formattedTimeTravel
            elif(i == 5):
                df.loc[index, '第五圈秒數'] = formattedTimeTravel

            i = i + 1
            prev = timestamp


        print("--------------End of " + cardIDfileName + "---------------")
    return df

In [8]:
def appendBestScoreAndWriteCSV(df):
    for k,v in bestScoreMap.items():
        # find the index of the tuple 
        i = df[df['卡號'] == cardIDMap[k]].index.values[0]
        # update the cell
        df.loc[i,'最佳圈秒數'] = formattedTime(bestScoreMap[k])
        
    df.to_csv('each_contest_after.csv', index=False)
    return df

In [9]:
df

,卡號,姓名,車隊,車型,日期
0,#1,Steven,team1,BMW M4,2019-03-02 11:16:07
1,#2,Michael,team2,McLaren 720S,2019-06-01 10:06:07
2,#3,Albert,team3,Porsche 918 GTR,2019-04-05 16:30:07


In [10]:
# Step3: Convert csv to xml
import csv

def writeToXML():
    csvFile = 'each_contest_after.csv'
    xmlFile = 'result.xml'

    csvData = csv.reader(open(csvFile))
    xmlData = open(xmlFile, 'w')
    xmlData.write('<?xml version="1.0"?>' + "\n")
    # there must be only one top-level tag
    xmlData.write('<csv_data>' + "\n")

    rowNum = 0
    for row in csvData:
        if rowNum == 0:
            tags = row
            # replace spaces w/ underscores in tag names
            for i in range(len(tags)):
                tags[i] = tags[i].replace(' ', '_')
        else: 
            xmlData.write('<row>' + "\n")
            for i in range(len(tags)):
                xmlData.write('    ' + '<' + tags[i] + '>' \
                              + row[i] + '</' + tags[i] + '>' + "\n")
            xmlData.write('</row>' + "\n")

        rowNum +=1

    xmlData.write('</csv_data>' + "\n")
    xmlData.close()

In [11]:
import time

apprx_finish_second = 5

start_time = time.time()
while(True):
    current_time = time.time()
    elapsed_time = current_time - start_time
    
    if elapsed_time > apprx_finish_second:
        break    
    
    df = readCSVAndparseOpFiles()
    df = appendBestScoreAndWriteCSV(df)
    print(df)
    writeToXML()
    

--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  20

   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04

--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2

--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2

--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2

...

--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2

--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  20

--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  20

--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  20

   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04-05 16:30:07    1  00:00.129   

       第二圈秒數      第三圈秒數      第四圈秒數      第五圈秒數      最佳圈秒數  
0  00:00.100  00:00.100  00:00.100  00:00.100  00:00.100  
1  00:00.150  00:00.050  00:00.170  00:00.180  00:00.050  
2  00:00.200  00:00.050        NaN        NaN  00:00.050  
--------------End of 1014E3_op---------------
10154D_op 不在此車競賽! 不做任何分析
--------------End of 10155D_op---------------
--------------End of 1014E9_op---------------
   卡號       姓名     車隊               車型                   日期  車競賽      第一圈秒數  \
0  #1   Steven  team1           BMW M4  2019-03-02 11:16:07    1  00:00.100   
1  #2  Michael  team2     McLaren 720S  2019-06-01 10:06:07    1  00:00.200   
2  #3   Albert  team3  Porsche 918 GTR  2019-04